# Read in data

In [1]:
from datetime import timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

airports = [
    "KATL",
    "KCLT",
    "KDEN",
    "KDFW",
    "KJFK",
    "KMEM",
    "KMIA",
    "KORD",
    "KPHX",
    "KSEA",
]

In [2]:
airport = "KDEN"

In [3]:
DATA_DIRECTORY = Path("code execution development data")

## LAMP

In [4]:
lamp = pd.read_csv(
    DATA_DIRECTORY / airport / f"{airport}_lamp.csv.bz2",
    parse_dates=["timestamp", "forecast_timestamp"],
)
lamp.head(5)

,timestamp,forecast_timestamp,temperature,wind_direction,wind_speed,wind_gust,cloud_ceiling,visibility,cloud,lightning_prob,precip
0,2020-11-14 00:30:00,2020-11-14 01:00:00,50,24,8,0,8.0,7,FW,N,False
1,2020-11-14 00:30:00,2020-11-14 02:00:00,47,23,9,0,8.0,7,FW,N,False
2,2020-11-14 00:30:00,2020-11-14 03:00:00,45,22,13,0,8.0,7,CL,N,False
3,2020-11-14 00:30:00,2020-11-14 04:00:00,43,22,15,0,8.0,7,FW,N,False
4,2020-11-14 00:30:00,2020-11-14 05:00:00,43,22,16,0,8.0,7,FW,N,False


## TBFM

In [5]:
tbfm = pd.read_csv(
    DATA_DIRECTORY / airport / f"{airport}_tbfm.csv.bz2",
    parse_dates=["timestamp", "scheduled_runway_estimated_time"],
)
tbfm.head(5)

,gufi,timestamp,scheduled_runway_estimated_time
0,UAL539.CLE.DEN.201112.2153.0048.TFM,2020-11-14,2020-11-14 01:15:07
1,SWA1002.MKE.DEN.201112.2313.0088.TFM,2020-11-14,2020-11-14 01:42:32
2,UAL1077.EWR.DEN.201112.2108.0106.TFM,2020-11-14,2020-11-14 01:22:02
3,SKW4752.CID.DEN.201112.2253.0165.TFM,2020-11-14,2020-11-14 01:10:54
4,SKW5234.GRR.DEN.201112.2218.0098.TFM,2020-11-14,2020-11-14 01:12:19


## ETD

In [6]:
etd = pd.read_csv(
    DATA_DIRECTORY / airport / f"{airport}_etd.csv.bz2",
    parse_dates=["departure_runway_estimated_time", "timestamp"],
)
etd.head(5)

,gufi,timestamp,departure_runway_estimated_time
0,SKW4678.DEN.FLG.201112.1843.0129.TFM,2020-11-13 18:00:05,2020-11-13 19:02:00
1,SKW4678.DEN.FLG.201112.1843.0129.TFM,2020-11-13 18:00:09,2020-11-13 19:07:00
2,SKW4678.DEN.FLG.201112.1843.0129.TFM,2020-11-13 18:00:10,2020-11-13 19:02:00
3,UAL459.DEN.MSP.201112.1638.0042.TFM,2020-11-13 18:00:14,2020-11-13 16:46:00
4,SWA6755.DEN.SMF.201112.1703.0087.TFM,2020-11-13 18:00:17,2020-11-13 17:26:00


## MFS

In [7]:
# TODO

# Initial Model, Linear Regression

In [42]:
submission_format = pd.read_csv(
    DATA_DIRECTORY / "test_labels.csv", parse_dates=["timestamp"]
)

In [43]:
df = submission_format[submission_format.airport == airport]
df.head(5)

,gufi,timestamp,airport,minutes_until_pushback
536,AAL1085.DEN.DFW.201114.1257.0267.TFM,2020-11-15 12:00:00,KDEN,56
537,AAL2222.DEN.MIA.201114.1217.0009.TFM,2020-11-15 11:15:00,KDEN,58
538,AAL2222.DEN.MIA.201114.1217.0009.TFM,2020-11-15 12:00:00,KDEN,13
539,ASA1177.DEN.SEA.201114.0132.0054.TFM,2020-11-15 00:45:00,KDEN,45
540,ASA741.DEN.SEA.201114.0023.0034.TFM,2020-11-15 00:00:00,KDEN,20


In [44]:
df.shape

(313, 4)

## Insert Data

In [45]:
# add empty columns
columns = ['etd',
           'traffic',
           'precip',
           'lightning_prob',
           'cloud',
           'visibility',
           'cloud_ceiling',
           'wind_gust',
           'wind_speed',
           'wind_direction',
           'temperature']

for col in columns:
    df[col] = ''
    
df.head(5)

C:\Users\Shantel\AppData\Local\Temp\ipykernel_33360\2179707400.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = ''
C:\Users\Shantel\AppData\Local\Temp\ipykernel_33360\2179707400.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = ''
C:\Users\Shantel\AppData\Local\Temp\ipykernel_33360\2179707400.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

,gufi,timestamp,airport,minutes_until_pushback,etd,traffic,precip,lightning_prob,cloud,visibility,cloud_ceiling,wind_gust,wind_speed,wind_direction,temperature
536,AAL1085.DEN.DFW.201114.1257.0267.TFM,2020-11-15 12:00:00,KDEN,56,,,,,,,,,,,
537,AAL2222.DEN.MIA.201114.1217.0009.TFM,2020-11-15 11:15:00,KDEN,58,,,,,,,,,,,
538,AAL2222.DEN.MIA.201114.1217.0009.TFM,2020-11-15 12:00:00,KDEN,13,,,,,,,,,,,
539,ASA1177.DEN.SEA.201114.0132.0054.TFM,2020-11-15 00:45:00,KDEN,45,,,,,,,,,,,
540,ASA741.DEN.SEA.201114.0023.0034.TFM,2020-11-15 00:00:00,KDEN,20,,,,,,,,,,,


In [46]:
def filter_lamp(current_time):
    # weather data
    valid_lamp = lamp.loc[(lamp.timestamp <= current_time) 
                    & (lamp.timestamp > valid_time) 
                    & (lamp.forecast_timestamp <= current_time) 
                    & (lamp.forecast_timestamp > valid_time)]
    return valid_lamp.iloc[-1]
    
def filter_tbfm(current_time):
    # traffic data
    valid_tbfm = tbfm.loc[(tbfm.timestamp <= current_time) 
                    & (tbfm.timestamp > valid_time) ]
    return valid_tbfm['gufi'].nunique()
    
def filter_etd(current_time):    
    #etd data
    valid_etd = etd.loc[(etd.timestamp <= current_time) 
                    & (etd.timestamp > valid_time) 
                    & (etd.gufi == row.gufi)]
    return (valid_etd.iloc[-1].departure_runway_estimated_time - current_time).total_seconds()

In [47]:
times = df.timestamp.unique()
for t in times:
    current_time = pd.to_datetime(t)
    indices = df[df.timestamp == t].index
    
    # insert etd data
    df.loc[indices, 'etd'] = filter_etd(current_time)
    
    # insert traffic data
    df.loc[indices, 'traffic'] = filter_tbfm(current_time)
    
    # insert weather data
    df.loc[indices, 'precip'] = filter_lamp(current_time).precip
    df.loc[indices, 'lightning_prob'] = filter_lamp(current_time).lightning_prob
    df.loc[indices, 'cloud'] = filter_lamp(current_time).cloud
    df.loc[indices, 'visibility'] = filter_lamp(current_time).visibility
    df.loc[indices, 'cloud_ceiling'] = filter_lamp(current_time).cloud_ceiling
    df.loc[indices, 'wind_gust'] = filter_lamp(current_time).wind_gust
    df.loc[indices, 'wind_speed'] = filter_lamp(current_time).wind_speed
    df.loc[indices, 'wind_direction'] = filter_lamp(current_time).wind_direction
    df.loc[indices, 'temperature'] = filter_lamp(current_time).temperature
    
df.head(5)

,gufi,timestamp,airport,minutes_until_pushback,etd,traffic,precip,lightning_prob,cloud,visibility,cloud_ceiling,wind_gust,wind_speed,wind_direction,temperature
536,AAL1085.DEN.DFW.201114.1257.0267.TFM,2020-11-15 12:00:00,KDEN,56,5160.0,559,False,N,FW,7,8.0,0,9,16,22
537,AAL2222.DEN.MIA.201114.1217.0009.TFM,2020-11-15 11:15:00,KDEN,58,7860.0,556,False,N,FW,7,8.0,0,8,28,32
538,AAL2222.DEN.MIA.201114.1217.0009.TFM,2020-11-15 12:00:00,KDEN,13,5160.0,559,False,N,FW,7,8.0,0,9,16,22
539,ASA1177.DEN.SEA.201114.0132.0054.TFM,2020-11-15 00:45:00,KDEN,45,45660.0,513,False,N,BK,7,7.0,29,21,31,33
540,ASA741.DEN.SEA.201114.0023.0034.TFM,2020-11-15 00:00:00,KDEN,20,48360.0,500,False,N,BK,7,7.0,29,21,31,33


## Feature Engineering

In [48]:
df.nunique()

gufi                      172
timestamp                  14
airport                     1
minutes_until_pushback     96
etd                        14
traffic                    12
precip                      1
lightning_prob              1
cloud                       4
visibility                  1
cloud_ceiling               2
wind_gust                   6
wind_speed                 10
wind_direction              6
temperature                 7
dtype: int64

In [49]:
# binary encoding
df.replace(False, 0, inplace=True)
df.replace(True, 0, inplace=True)

C:\Users\Shantel\AppData\Local\Temp\ipykernel_33360\3913201293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(False, 0, inplace=True)
C:\Users\Shantel\AppData\Local\Temp\ipykernel_33360\3913201293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(True, 0, inplace=True)


In [50]:
# nominal encoding
df = pd.get_dummies(df, columns=['cloud', 'lightning_prob'], drop_first=True)
df.head(5)

,gufi,timestamp,airport,minutes_until_pushback,etd,traffic,precip,visibility,cloud_ceiling,wind_gust,wind_speed,wind_direction,temperature,cloud_CL,cloud_FW,cloud_SC
536,AAL1085.DEN.DFW.201114.1257.0267.TFM,2020-11-15 12:00:00,KDEN,56,5160.0,559,0,7,8.0,0,9,16,22,0,1,0
537,AAL2222.DEN.MIA.201114.1217.0009.TFM,2020-11-15 11:15:00,KDEN,58,7860.0,556,0,7,8.0,0,8,28,32,0,1,0
538,AAL2222.DEN.MIA.201114.1217.0009.TFM,2020-11-15 12:00:00,KDEN,13,5160.0,559,0,7,8.0,0,9,16,22,0,1,0
539,ASA1177.DEN.SEA.201114.0132.0054.TFM,2020-11-15 00:45:00,KDEN,45,45660.0,513,0,7,7.0,29,21,31,33,0,0,0
540,ASA741.DEN.SEA.201114.0023.0034.TFM,2020-11-15 00:00:00,KDEN,20,48360.0,500,0,7,7.0,29,21,31,33,0,0,0


# Linear Regression

## Model 1

In [98]:
import statsmodels.api as sm

Y = df['minutes_until_pushback']
X = df.drop(columns=['gufi', 'timestamp', 'airport', 'minutes_until_pushback'])
X = sm.add_constant(X)

model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     minutes_until_pushback   R-squared:                       0.308
Model:                                OLS   Adj. R-squared:                  0.288
Method:                     Least Squares   F-statistic:                     14.99
Date:                    Fri, 31 Mar 2023   Prob (F-statistic):           3.90e-20
Time:                            23:17:04   Log-Likelihood:                -1433.8
No. Observations:                     313   AIC:                             2888.
Df Residuals:                         303   BIC:                             2925.
Df Model:                               9                                         
Covariance Type:                nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
etd               -0.0042      0.001     -3.479      0.001      -0.007      -0.002
traffic           -3.2417      0.413     -7.843      0.000      -4.055      -2.428
precip          1.504e-13   3.04e-14      4.941      0.000    9.05e-14     2.1e-13
visibility       192.4646     28.552      6.741      0.000     136.279     248.650
cloud_ceiling     89.2936     11.724      7.616      0.000      66.223     112.364
wind_gust          1.3653      0.613      2.228      0.027       0.160       2.571
wind_speed        -1.3936      1.187     -1.174      0.241      -3.729       0.942
wind_direction    10.3820      3.993      2.600      0.010       2.524      18.240
temperature      -12.9021      4.897     -2.635      0.009     -22.538      -3.266
cloud_CL         -38.0168     17.091     -2.224      0.027     -71.649      -4.385
cloud_FW         -41.6222     12.049     -3.454      0.001     -65.333     -17.911
cloud_SC         -23.5320      5.453     -4.315      0.000     -34.263     -12.801
==============================================================================
Omnibus:                       79.447   Durbin-Watson:                   1.846
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              230.208
Skew:                           1.140   Prob(JB):                     1.03e-50
Kurtosis:                       6.529   Cond. No.                     3.56e+20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.38e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [99]:
# calculate RMSE
from statsmodels.tools.eval_measures import rmse

ypred = round(model.predict(X))
RMSE = rmse(Y, ypred)
RMSE

23.60822405034952

In [100]:
# check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# create VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
vif_data

C:\Users\Shantel\.conda\envs\myenv\lib\site-packages\statsmodels\regression\linear_model.py:1752: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\Shantel\.conda\envs\myenv\lib\site-packages\statsmodels\regression\linear_model.py:1752: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\Shantel\.conda\envs\myenv\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
0,etd,81.513999
1,traffic,30.617194
2,precip,NaN
3,visibility,0.000000
4,cloud_ceiling,inf
5,wind_gust,16.746364
6,wind_speed,10.970848
7,wind_direction,90.012153
8,temperature,85.263066
9,cloud_CL,inf


## Model 2

In [110]:
# reduce features
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector

# re-define independent variables
X = df.drop(columns=['gufi', 'timestamp', 'airport', 'minutes_until_pushback'])

# linear regression estimator to be used in feature selection
reg = LinearRegression().fit(X, Y)

In [112]:
# reduce features using SequentialFeatureSelector
sfs = SequentialFeatureSelector(reg, n_features_to_select=4)
sfs.fit(X, Y)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=4)

In [113]:
# return selected features
sfs.get_feature_names_out()

array(['etd', 'traffic', 'cloud_ceiling', 'wind_speed'], dtype=object)

In [114]:
# re-fit model
X = df[['etd', 'traffic', 'cloud_ceiling', 'wind_speed']]
X = sm.add_constant(X)

model2 = sm.OLS(Y,X).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     minutes_until_pushback   R-squared:                       0.268
Model:                                OLS   Adj. R-squared:                  0.258
Method:                     Least Squares   F-statistic:                     28.18
Date:                    Fri, 31 Mar 2023   Prob (F-statistic):           5.89e-20
Time:                            23:22:35   Log-Likelihood:                -1442.6
No. Observations:                     313   AIC:                             2895.
Df Residuals:                         308   BIC:                             2914.
Df Model:                               4                                         
Covariance Type:                nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           954.3191    121.094      7.881      0.000     716.042    1192.596
etd              -0.0014      0.000     -3.560      0.000      -0.002      -0.001
traffic          -2.2808      0.294     -7.751      0.000      -2.860      -1.702
cloud_ceiling    50.6003      6.954      7.276      0.000      36.916      64.285
wind_speed       -2.1175      0.609     -3.479      0.001      -3.315      -0.920
==============================================================================
Omnibus:                       78.915   Durbin-Watson:                   1.871
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              199.855
Skew:                           1.188   Prob(JB):                     4.00e-44
Kurtosis:                       6.111   Cond. No.                     3.69e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.69e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [115]:
# re-calculate rmse
ypred = round(model2.predict(X))
RMSE = rmse(Y, ypred)
RMSE

24.296820023591273

In [116]:
# re-check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# create VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
vif_data

,feature,VIF
0,const,7655.702391
1,etd,9.022144
2,traffic,14.908657
3,cloud_ceiling,6.265623
4,wind_speed,2.773062
